In [ ]:
from typing import Dict, List, Annotated
import numpy as np
import os
from sklearn.cluster import KMeans
import pickle



DB_SEED_NUMBER = 42
ELEMENT_SIZE = np.dtype(np.float32).itemsize
DIMENSION = 70
class CustomIndex: pass

class VecDB:
    def __init__(self, database_file_path = "saved_db.dat", index_file_path = "index.dat", new_db = True, db_size = None) -> None:
        self.db_path = database_file_path
        self.index_path = index_file_path
        if new_db:
            if db_size is None:
                raise ValueError("You need to provide the size of the database")
            # delete the old DB file if exists
            if os.path.exists(self.db_path):
                os.remove(self.db_path)
            self.generate_database(db_size)
    
    def generate_database(self, size: int) -> None:
        rng = np.random.default_rng(DB_SEED_NUMBER)
        vectors = rng.random((size, DIMENSION), dtype=np.float32)
        self._write_vectors_to_file(vectors)
        self._build_index()
    # np.memmap() Parameters:
    # First argument: File path to store data
    # dtype: Data type (float32 here)
    # mode: File access mode
    # 'w+': Read/write, create if not exists
    # shape: Dimensions of the array
    # returns file contents on disk
    def _write_vectors_to_file(self, vectors: np.ndarray) -> None:
        mmap_vectors = np.memmap(self.db_path, dtype=np.float32, mode='w+', shape=vectors.shape)
        mmap_vectors[:] = vectors[:]
        mmap_vectors.flush()

    def _get_num_records(self) -> int:
        return os.path.getsize(self.db_path) // (DIMENSION * ELEMENT_SIZE)

    def insert_records(self, rows: Annotated[np.ndarray, (int, 70)]):
        num_old_records = self._get_num_records()
        num_new_records = len(rows)
        full_shape = (num_old_records + num_new_records, DIMENSION)
        mmap_vectors = np.memmap(self.db_path, dtype=np.float32, mode='r+', shape=full_shape)
        mmap_vectors[num_old_records:] = rows
        mmap_vectors.flush()
        #TODO: might change to call insert in the index, if you need
        self._build_index()

    def get_one_row(self, row_num: int) -> np.ndarray:
        # This function is only load one row in memory
        try:
            offset = row_num * DIMENSION * ELEMENT_SIZE
            # [0] is necessary because:
            # mmap_vector is 2D array: [[x1, x2, ..., x70]]
            # [0] extracts the single row as 1D: [x1, x2, ..., x70]
            mmap_vector = np.memmap(self.db_path, dtype=np.float32, mode='r', shape=(1, DIMENSION), offset=offset)
            return np.array(mmap_vector[0])
        except Exception as e:
            return f"An error occurred: {e}"

    def get_all_rows(self) -> np.ndarray:
        # Take care this load all the data in memory
        num_records = self._get_num_records()
        vectors = np.memmap(self.db_path, dtype=np.float32, mode='r', shape=(num_records, DIMENSION))
        return np.array(vectors)
    
    def retrieve(self, query: Annotated[np.ndarray, (1, DIMENSION)], top_k = 5):
        scores = []
        num_records = self._get_num_records()
        file = open(self.index_path,'rb')
        index = pickle.load(file)
        file.close()

        cluster_centers=index.centroids
        labels_list=index.labels_list
        for i,vec in enumerate(cluster_centers):
            score=self._cal_score(query,vec)
            scores.append((score,i))
        # # here we assume that the row number is the ID of each vector
        # for row_num in range(num_records):
        #     vector = self.get_one_row(row_num)
        #     score = self._cal_score(query, vector)
        #     scores.append((score, row_num))
        # here we assume that if two rows have the same score, return the lowest ID
        scores = sorted(scores, reverse=True)[:top_k]
        top_vector=[]
        # for item in scores:
        top_vector.append(labels_list[scores[0][1]])
        return top_vector
    
    def _cal_score(self, vec1, vec2):
        dot_product = np.dot(vec1, vec2)
        norm_vec1 = np.linalg.norm(vec1)
        norm_vec2 = np.linalg.norm(vec2)
        cosine_similarity = dot_product / (norm_vec1 * norm_vec2)
        return cosine_similarity

    def _build_index(self, num_clusters=100, num_subspaces=7):
            
            # Placeholder for index building logic
            ### IVF
            # Apply k means clustering to all vectors -> return the cluster centroids 
            # store the clusters and their centroids in a array or dictionary?
            # Within each cluster:
            # loop over each vector 
            # create an array of subspace arrays (parameter) subspaces[[   [subvector1 of array 1],[subvector1 of array 2]   ] , [],...]
            # apply k-means clustering for each subspace[0] subspace[1] etc..
            # Generate the codebook
            
            
            # Build the PQ-IVF index.
            # Args:
            #     num_clusters (int): Number of clusters for the inverted file (IVF).
            #     num_subspaces (int): Number of subspaces for product quantization.
            #     subspace_dim (int): Dimension of each subspace.
            # Sample dataset
            vectors = self.get_all_rows()

            # Step 1: Coarse Quantization (Clustering)
            kmeans = KMeans(n_clusters=10)
            labels = kmeans.fit_predict(vectors)  # Assign each vector to a cluster
            cluster_centers = kmeans.cluster_centers_
            # Step 2: Construct Posting Lists
            labels_list = {i: [] for i in range(10)}  # Two clusters: 0 and 1
            for i, label in enumerate(labels):
                labels_list[label].append(i)
            
            CustomIndex.centroids=cluster_centers
            CustomIndex.labels_list=labels_list
            
            filehandler = open(self.index_path,"wb")
            pickle.dump(CustomIndex,filehandler)
            filehandler.close()

            file = open(self.index_path,'rb')
            object_file = pickle.load(file)
            file.close()

            print(object_file.centroids, object_file.labels_list, sep=', ')
            # Print Clustering Results
            print("Cluster Centers:", cluster_centers)
            print("Labels:", labels)
            print("Posting Lists:", labels_list)

            # # # Query Processing
            # query = np.random.randint(0, 10, (1, 7))  # Integers between 0 and 9
            # nearest_centroid = np.argmin([np.linalg.norm(query - centroid) for centroid in kmeans.cluster_centers_])

            # # Fine Search (Within the posting list of nearest centroid)
            # nearest_vectors = [vectors[i] for i in labels_list[nearest_centroid]]
            # closest_vector = min(nearest_vectors, key=lambda x: np.linalg.norm(query - x))
            # print(f"Actual vector: {query}")
            # print(f"Nearest vector: {closest_vector}")
    

In [58]:
# Create an instance of VecDB and random DB of size 10K
db = VecDB(db_size = 10**3)

# Retrieve similar images for a given query
query_vector = np.random.rand(70) # Query vector of dimension 70
similar_images = db.retrieve(query_vector, top_k=5)
print(similar_images)

print(db.get_one_row(similar_images[0]))


d:\Programs\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
d:\Programs\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


[[0.43187883 0.47953948 0.59312284 0.51649666 0.5528442  0.6288911
  0.61080796 0.36431763 0.5185671  0.42126635 0.52380705 0.5567874
  0.4839619  0.5286623  0.4352572  0.47157943 0.47131154 0.56470335
  0.6302961  0.537825   0.37809974 0.5138853  0.70083207 0.6480595
  0.5129867  0.3498488  0.39060143 0.43483418 0.6169812  0.5358724
  0.5992363  0.4749908  0.374883   0.5177031  0.52992076 0.4877657
  0.5008563  0.5430082  0.6113124  0.6271156  0.40733722 0.5960099
  0.5131988  0.5418098  0.6339903  0.5182667  0.4143639  0.6351447
  0.38917688 0.44540942 0.51121217 0.50888115 0.41288173 0.52871794
  0.5001786  0.53358984 0.4973614  0.49575827 0.58931005 0.5123297
  0.5515596  0.36735308 0.59534794 0.5151511  0.5949171  0.5432087
  0.521006   0.39981484 0.5544974  0.5250895 ]
 [0.49499708 0.46210396 0.5207688  0.6439846  0.5247751  0.5852406
  0.42157978 0.465078   0.5350113  0.4313518  0.46580195 0.49424586
  0.40427485 0.48771155 0.42297724 0.46548316 0.4521501  0.45619315
  0.4539351

IndexError: tuple index out of range

In [ ]:
print(db._cal_score(query_vector,db.get_one_row((similar_images[0]))))
print(db._cal_score(query_vector,db.get_one_row((similar_images[1]))))
print(db._cal_score(query_vector,db.get_one_row((similar_images[2]))))
print(db._cal_score(query_vector,db.get_one_row((similar_images[3]))))
print(db._cal_score(query_vector,db.get_one_row((similar_images[4]))))

0.8306092778089047
0.8243485783299538
0.8196922152617356
0.8154010774167503
0.8137924911889398


In [45]:

# Example usage
db =VecDB(db_size=1000)
query_vector = np.array([1, 2, 3])
similar_image_vector = np.array([4, 5, 6])

similarity = db._cal_score(query_vector, similar_image_vector)
print("Cosine Similarity Score:", similarity)

d:\Programs\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
d:\Programs\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


Cosine Similarity Score: 0.9746318461970762


In [ ]:
import random

#consider this as a high dimensional vector
vec = v = [random.randint(1,20) for i in range(70)]
print(vec)

[7, 2, 7, 4, 16, 18, 8, 10, 5, 1, 18, 7, 9, 13, 9, 15, 14, 2, 12, 13, 20, 4, 19, 16, 13, 13, 4, 15, 7, 19, 19, 18, 15, 15, 20, 18, 12, 14, 3, 8, 4, 14, 9, 12, 2, 19, 10, 15, 7, 2, 12, 6, 20, 4, 13, 17, 18, 17, 19, 3, 14, 19, 9, 7, 15, 13, 11, 19, 7, 15]


In [ ]:
chunk_count = 35
vector_size = len(vec)

# vector_size must be divisable by chunk_size
assert vector_size % chunk_count == 0
# length of each subvector will be vector_size/ chunk_count
subvector_size = int(vector_size / chunk_count)

# subvectors
sub_vectors = [vec[row: row+subvector_size] for row in range(0, vector_size, subvector_size)]
sub_vectors

[[7, 2],
 [7, 4],
 [16, 18],
 [8, 10],
 [5, 1],
 [18, 7],
 [9, 13],
 [9, 15],
 [14, 2],
 [12, 13],
 [20, 4],
 [19, 16],
 [13, 13],
 [4, 15],
 [7, 19],
 [19, 18],
 [15, 15],
 [20, 18],
 [12, 14],
 [3, 8],
 [4, 14],
 [9, 12],
 [2, 19],
 [10, 15],
 [7, 2],
 [12, 6],
 [20, 4],
 [13, 17],
 [18, 17],
 [19, 3],
 [14, 19],
 [9, 7],
 [15, 13],
 [11, 19],
 [7, 15]]

In [ ]:
# subvectors are then processed and linked to their closest centroids, also known as reproduction values, within the respective subclusters.

k = 56
assert k % chunk_count == 0
k_ = int(k/chunk_count)

from random import randint
# reproduction values
c = []  
for j in range(chunk_count):
    # each j represents a subvector position
    c_j = []
    for i in range(k_):
        # each i represents a cluster/reproduction value position 
       c_ji = [randint(0, 9) for _ in range(subvector_size)]
       c_j.append(c_ji)  # add cluster centroid to subspace list
    
  # add subspace list of centroids
    c.append(c_j)

AssertionError: 

In [ ]:
#helper function to calculate euclidean distance
def euclidean(v, u):
    distance = sum((x - y) ** 2 for x, y in zip(v, u)) ** .5
    return distance

#helper function to create unique ids
def nearest(c_j, chunk_j):
    distance = 9e9
    for i in range(k_):
        new_dist = euclidean(c_j[i], chunk_j)
        if new_dist < distance:
            nearest_idx = i
            distance = new_dist
    return nearest_idx

In [ ]:
ids = []
# unique centroid IDs for each subvector
for j in range(chunk_count):
    i = nearest(c[j], sub_vectors[j])
    ids.append(i)
print(ids)

[1, 0, 2, 2, 0, 2, 0, 0, 3, 0, 1, 3, 0, 1]


In [ ]:
quantized = []
for j in range(chunk_count):
    c_ji = c[j][ids[j]]
    quantized.extend(c_ji)

print(quantized)

[8, 4, 0, 2, 9, 7, 4, 7, 1, 8, 3, 0, 9, 8, 7, 7, 8, 7, 4, 3, 5, 9, 9, 7, 5, 5, 7, 7, 8, 4, 6, 9, 5, 4, 9, 9, 0, 9, 0, 8, 0, 8, 8, 9, 3, 5, 6, 4, 3, 3, 6, 9, 4, 2, 4, 8, 6, 8, 5, 9, 6, 8, 9, 9, 0, 0, 4, 9, 5, 9]


In [ ]:
from sklearn.cluster import KMeans
import numpy as np

# Sample dataset
vectors = np.random.randint(0, 10, (10, 7))  # Integers between 0 and 9

print(vectors)
# Step 1: Coarse Quantization (Clustering)
kmeans = KMeans(n_clusters=2)
labels = kmeans.fit_predict(vectors)  # Assign each vector to a cluster
cluster_centers = kmeans.cluster_centers_

# Step 2: Construct Posting Lists
posting_lists = {i: [] for i in range(2)}  # Two clusters: 0 and 1
for i, label in enumerate(labels):
    posting_lists[label].append(i)
    

# Print Clustering Results
print("Cluster Centers:", cluster_centers)
print("Labels:", labels)
print("Posting Lists:", posting_lists)

# Query Processing
query = np.random.randint(0, 10, (1, 7))  # Integers between 0 and 9
nearest_centroid = np.argmin([np.linalg.norm(query - centroid) for centroid in kmeans.cluster_centers_])

# Fine Search (Within the posting list of nearest centroid)
nearest_vectors = [vectors[i] for i in posting_lists[nearest_centroid]]
closest_vector = min(nearest_vectors, key=lambda x: np.linalg.norm(query - x))
print(f"Actual vector: {query}")
print(f"Nearest vector: {closest_vector}")

[[6 1 2 9 3 6 4]
 [2 9 6 6 1 1 8]
 [6 8 0 4 3 9 3]
 [5 5 4 8 8 1 7]
 [2 2 4 7 3 8 4]
 [8 2 9 7 9 1 5]
 [0 7 9 4 3 1 5]
 [6 3 4 2 0 5 0]
 [8 4 9 2 0 6 5]
 [5 4 1 3 0 2 9]]
Cluster Centers: [[3.75       5.75       7.         6.25       5.25       1.
  6.25      ]
 [5.5        3.66666667 3.33333333 4.5        1.5        6.
  4.16666667]]
Labels: [1 0 1 0 1 0 0 1 1 1]
Posting Lists: {0: [1, 3, 5, 6], 1: [0, 2, 4, 7, 8, 9]}
Actual vector: [[6 6 6 5 8 1 4]]
Nearest vector: [5 5 4 8 8 1 7]


d:\Programs\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
d:\Programs\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

class ProductQuantization:
    def __init__(self, m, k):
        """
        Initialize the Product Quantization class.
        :param m: Number of subspaces (split vector into m parts)
        :param k: Number of clusters per subspace
        """
        self.m = m
        self.k = k
        self.codebooks = []

    def fit(self, data):
        """
        Fit the quantizer to the data.
        :param data: Dataset of shape (n_samples, d_features)
        """
        n_samples, d_features = data.shape
        assert d_features % self.m == 0, "Number of features must be divisible by m"
        
        self.subvector_dim = d_features // self.m
        self.codebooks = []
        
        for i in range(self.m):
            sub_data = data[:, i * self.subvector_dim:(i + 1) * self.subvector_dim]
            kmeans = KMeans(n_clusters=self.k, random_state=42).fit(sub_data)
            self.codebooks.append(kmeans.cluster_centers_)
    
    def encode(self, data): #build the index
        """
        Encode the dataset into quantized indices.
        :param data: Dataset of shape (n_samples, d_features)
        :return: Encoded indices of shape (n_samples, m)
        """
        n_samples, d_features = data.shape
        codes = np.zeros((n_samples, self.m), dtype=np.int32) # Stores the nearest centroid for each subspace: if m = 4 -> [[1,5,1,6],[9,1,5,3],...]
        
        for i in range(self.m):
            sub_data = data[:, i * self.subvector_dim:(i + 1) * self.subvector_dim] #partition the data into subspaces
            distances = cdist(sub_data, self.codebooks[i]) #
            codes[:, i] = np.argmin(distances, axis=1)
        
        return codes #codes contain the compressed representation of the data
    
    def decode(self, codes): #used 
        """
        Decode the quantized indices back to approximate vectors.
        :param codes: Encoded indices of shape (n_samples, m)
        :return: Approximate vectors of shape (n_samples, d_features)
        """
        n_samples = codes.shape[0]
        decoded_vectors = np.zeros((n_samples, self.m * self.subvector_dim))
        
        for i in range(self.m):
            decoded_vectors[:, i * self.subvector_dim:(i + 1) * self.subvector_dim] = self.codebooks[i][codes[:, i]]
        
        return decoded_vectors

    def search(self, query, codes, top_k=1):
        """
        Perform approximate nearest neighbor search.
        :param query: Query vector of shape (1, d_features)
        :param codes: Encoded indices of the dataset
        :param top_k: Number of nearest neighbors to return
        :return: Indices of top_k nearest neighbors
        """
        distances = np.zeros(codes.shape[0])
        
        for i in range(self.m):
            sub_query = query[:, i * self.subvector_dim:(i + 1) * self.subvector_dim]
            sub_distances = cdist(sub_query, self.codebooks[i])
            distances += sub_distances[0, codes[:, i]]
        
        return np.argsort(distances)[:top_k]

# Example usage
if __name__ == "__main__":
    # Generate synthetic data
    data = np.random.random((1000, 16))  # 1000 samples, 16 dimensions
    
    query = np.random.random((1, 16))   # 1 query vector
    print(query)
    pq = ProductQuantization(m=8, k=256)  # 4 subspaces, 256 clusters per subspace
    pq.fit(data)
    
    codes = pq.encode(data)
    reconstructed_data = pq.decode(codes)
    
    # Perform search
    neighbors = pq.search(query, codes, top_k=5)
    print(data)
    print(codes)
    print(reconstructed_data)
    print("Nearest neighbors:", neighbors)


[[0.94669777 0.14702176 0.03122288 0.93221853 0.8269393  0.78845497
  0.30489522 0.9944374  0.97145154 0.72643284 0.11840847 0.83041497
  0.25410993 0.29975197 0.16046107 0.2680251 ]]


d:\Programs\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
d:\Programs\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
d:\Programs\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
d:\Programs\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks

[[0.09518839 0.35055304 0.9398079  ... 0.95536698 0.62127548 0.25900692]
 [0.60531046 0.0299943  0.18020337 ... 0.84442837 0.30599367 0.31717393]
 [0.12436448 0.70804765 0.40053623 ... 0.01026024 0.98808191 0.81107594]
 ...
 [0.83905272 0.73320902 0.38375223 ... 0.27363777 0.37699509 0.06085107]
 [0.06235347 0.81673454 0.83788639 ... 0.13500675 0.61006013 0.20705075]
 [0.5731672  0.76549188 0.87305659 ... 0.68015527 0.97821466 0.63104061]]
[[ 55 168  80 ...  63  69 117]
 [157 240 141 ... 103 160 169]
 [111 134  58 ...  23  49   5]
 ...
 [227  34 191 ... 211 147  18]
 [ 77  66 245 ... 150 134 118]
 [ 51 120  81 ... 228 224  70]]
[[0.09794463 0.35227004 0.91964504 ... 0.95014297 0.62753523 0.24410977]
 [0.61093234 0.01673905 0.15575159 ... 0.85667647 0.30498966 0.29733311]
 [0.12194592 0.71991971 0.39669213 ... 0.01502664 0.98060759 0.81912355]
 ...
 [0.82988276 0.71941122 0.39528657 ... 0.29586202 0.37226255 0.06036116]
 [0.07032296 0.81166198 0.84902497 ... 0.12305148 0.57763475 0.1852